### A  demonstration of sims_movingObjects ###

This notebook is also available in the [sims_movingObjects](https://github.com/lsst/sims_movingObjects) github repository, in the examples directory:
https://github.com/lsst/sims_movingObjects/blob/master/examples/Demo%20sims_movingObjects.ipynb

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import lsst.sims.movingObjects as mo
import lsst.sims.maf.db as db
from lsst.sims.maf.batches import getColMap

For most typical use of sims_movingObjects, you just need to use the script 'makeLSSTobs.py' :

```
(lsst-scipipe) [ewok:~] lynnej% makeLSSTobs.py --help

usage: makeLSSTobs.py [-h] [--orbitFile ORBITFILE] [--outDir OUTDIR]
                      [--obsFile OBSFILE] [--sqlConstraint SQLCONSTRAINT]
                      [--noCamera] [--rFov RFOV]
                      [--interpolation INTERPOLATION] [--obscode OBSCODE]
                      [--tStep TSTEP] [--ephMode EPHMODE]
                      opsimDb

Generate moving object detections.

positional arguments:
  opsimDb               Opsim run db file

optional arguments:
  -h, --help            show this help message and exit
  --orbitFile ORBITFILE
                        File containing the moving object orbits.
  --outDir OUTDIR       Output directory for moving object detections.
  --obsFile OBSFILE     Output file name for moving object observations.
                        Default will build outDir/opsimRun_orbitFile_obs.txt.
  --sqlConstraint SQLCONSTRAINT
                        SQL constraint to use to select data from opsimDb.
                        Default no constraint.
  --noCamera            Do not use the LSST camera footprint, use rFov
                        instead.
  --rFov RFOV           If not using camera footprint, use a circular FOV with
                        radius rFOV degrees. Default 1.75 degrees.
  --interpolation INTERPOLATION
                        Type of interpolation between ephemerides to use.
                        Options include linear, chebyshev, and direct (no
                        interpolation). Default is linear.
  --obscode OBSCODE     Obscode for generating observations with linear or
                        direct interpolation. Default is I11 (Cerro Pachon).
  --tStep TSTEP         Timestep between ephemeris generation / linear
                        interpolation steps (in days). Relevant for linear
                        interpolation only! Default 2 hours.
  --ephMode EPHMODE     2body or nbody mode for ephemeris generation. Default
                        is nbody.
```

This runs the steps below, as a whole, to produce an output file with the observations.


But let's break down a little bit of what's happening behind the scenes.

### Reading the orbit file.

In [3]:
# First the orbit file - this is read into an Orbits object.
orbitDir = '.'
orbitfile = 'neos_100.s3m'

orbits = mo.Orbits()
orbits.readOrbits(os.path.join(orbitDir, orbitfile))

In [4]:
# The actual orbits are checked to see if they contain required columns: 
for orbit_type in orbits.dataCols:
    print(orbit_type, orbits.dataCols[orbit_type])

COM ['objId', 'q', 'e', 'inc', 'Omega', 'argPeri', 'tPeri', 'epoch', 'H', 'g', 'sed_filename']
KEP ['objId', 'a', 'e', 'inc', 'Omega', 'argPeri', 'meanAnomaly', 'epoch', 'H', 'g', 'sed_filename']
CART ['objId', 'x', 'y', 'z', 'xdot', 'ydot', 'zdot', 'epoch', 'H', 'g', 'sed_filename']


In [5]:
# and if 'standard but not quite the same' orbit column names are used, 
# the column names are swapped to these standard names. 

# Note that columns 'objId' = anything (including strings), and columns H / g / sed_filename are not required.
# If H is not specified, the value of 20 will be used
# If g is not specified, the value of 0.15 will be used
# If sed_filename is not specified (it should match a filename in $SIMS_MOVINGOBJECTS_DIR/data), one will be assigned.

help(orbits.assignSed)

Help on method assignSed in module lsst.sims.movingObjects.orbits:

assignSed(orbits, randomSeed=None) method of lsst.sims.movingObjects.orbits.Orbits instance
    Assign either a C or S type SED, depending on the semi-major axis of the object.
    P(C type) = 0 (a<2); 0.5*a - 1 (2<a<4); 1 (a > 4),
    based on figure 23 from Ivezic et al 2001 (AJ, 122, 2749).
    
    Parameters
    ----------
    orbits : pandas.DataFrame, pandas.Series or numpy.ndarray
       Array-like object containing orbital parameter information.
    
    Returns
    -------
    numpy.ndarray
        Array containing the SED type for each object in 'orbits'.



In [7]:
# The final orbit data is kept in a class attribute: orbits:  
# note that additional data in the orbit file (such as otype) is retained.
orbits.orbits[0:5]

,objId,otype,q,e,inc,Omega,argPeri,tPeri,H,g,epoch,model,sed_filename
0,S1000000,S3M,3.01822,0.05208,22.56035,211.00286,335.42134,57801.64493,14.20,0.15,57231.0,V150728,C.dat
1,S1000001,S3M,2.10974,0.07518,4.91571,209.40298,322.66447,56722.78779,20.57,0.15,57231.0,V150728,S.dat
2,S1000002,S3M,2.80523,0.07777,1.24945,112.52284,139.86858,56406.44304,14.65,0.15,57231.0,V150728,C.dat
3,S1000003,S3M,2.10917,0.13219,1.46615,266.54621,232.24412,56980.11676,19.58,0.15,57231.0,V150728,C.dat
4,S1000004,S3M,2.17676,0.19949,12.92422,162.14580,192.22312,56808.90252,10.56,0.15,57231.0,V150728,S.dat


### Connecting to (and reading from) the opsim data file.

In [8]:
# Look at the MAF documentation for more information on the lsst.sims.maf.db.OpsimDatabase class and methods.
opsimDir = '/Users/lynnej/opsim/db'
opsimrun = 'baseline2018b'
opsdb = db.OpsimDatabase(os.path.join(opsimDir, opsimrun + '.db'))
colmap = getColMap(opsdb)

In [9]:
# Basically, we will just read all of these requested columns from the database: (for now we'll stick to night < 100)
reqcols = [colmap['mjd'], colmap['night'], colmap['ra'], colmap['dec'],
               'rotSkyPos', colmap['filter'], colmap['exptime'], colmap['seeingEff'],
               colmap['seeingGeom'], colmap['fiveSigmaDepth'], 'solarElong']
degreesIn = colmap['raDecDeg']
print(reqcols)

['observationStartMJD', 'night', 'fieldRA', 'fieldDec', 'rotSkyPos', 'filter', 'visitExposureTime', 'seeingFwhmEff', 'seeingFwhmGeom', 'fiveSigmaDepth', 'solarElong']


In [10]:
simdata = opsdb.fetchMetricData(reqcols, sqlconstraint='night < 100')

In [11]:
# For now, we apply a bit of a hack fix to standardize some column names:
simdata = mo.fixObsData(simdata, degreesIn=degreesIn)

In [12]:
# Simdata is a numpy recarray - let's just use pandas to make it easier to read here.
pd.DataFrame(simdata[0:3])

,observationStartMJD,night,fieldRA,fieldDec,rotSkyPos,filter,visitExposureTime,seeingFwhmEff,seeingFwhmGeom,fiveSigmaDepth,solarElong,ra,dec
0,59853.016794,1,305.088793,-24.889283,180.000000,z,30.0,0.692029,0.620848,23.348066,113.652908,305.088793,-24.889283
1,59853.018021,1,304.770694,35.966846,180.786087,z,30.0,1.182104,1.023690,22.308947,113.957998,304.770694,35.966846
2,59853.020185,1,305.482200,-62.802603,181.134222,z,30.0,0.766303,0.681901,23.178827,99.684211,305.482200,-62.802603


### Generating observations. 

There is more than one way to actually generate observations, but for now let's look at the linear interpolation approach (what I'd recommend at the moment). This is implemented in the class "LinearObs", which provides higher level methods that basically do the following: 

With LinearObs generating observations for each object goes as follows:  
* generate ephemerides between the start and end of the survey data, on stepsizes of tStep (typically, 2 hours)
* make linear interpolations between each of these steps, for every parameter returned by the ephemeris generation
* calculate the distance between the location of the object and the field pointing, for each observation
* if the distance is less than the specified 'rFov': count this observation as "observed" (if using the camera footprint, then the detailed camera footprint is overlaid for observations, and only objects which actually lie on active silicon are counted as "observed")
* for the pointings where the object was observed, then calculate trailing losses and write the output to disk (including a color term, appropriate for the filter of each observation).

(Note: there's plenty of room to improve on the procedure above, which could both speed things up and improve accuracy .. if this is something you're interested in contributing to, please consider writing a new Obs class, preferably inheriting from the BaseObs class).


In [13]:
# Using the "LinearObs" class.
usecamerafootprint = False
if usecamerafootprint:
    # Use the actual camera footprint.
    cameraFootprint = mo.LsstCameraFootprint()
else:
    # Or not (just use a circular fov)
    cameraFootprint = None
rFov = 1.75  # in degrees
obscode = 'I11'
ephMode = 'nbody' # nbody or 2body
obs = mo.LinearObs(cameraFootprint=cameraFootprint, rFov=rFov, obscode=obscode, timescale='TAI', ephMode=ephMode, 
                   timeCol=colmap['mjd'], seeingCol=colmap['seeingGeom'], visitExpTimeCol=colmap['exptime'])

In [14]:
# Set the orbits.
obs.setOrbits(orbits)
# Read the filter curves (to calculate the colors for each object)
filterlist = np.unique(simdata[colmap['filter']])
obs.readFilters(filterlist=filterlist)
# Calculate all colors ahead of time - this is just to create a dictionary early.
sednames = np.unique(obs.orbits.orbits['sed_filename'])
for sedname in sednames:
    obs.calcColors(sedname)
# Want to see the lsst colors?  (these are all x-V band colors)
obs.colors

{'C.dat': {'g': 0.28041793225569478,
  'i': -0.2926196728132453,
  'r': -0.177293845688272,
  'u': 1.5442111906705982,
  'y': -0.3028771664514025,
  'z': -0.29808899160965296},
 'S.dat': {'g': 0.36850606460945201,
  'i': -0.45720772665728049,
  'r': -0.26270911487248227,
  'u': 1.8626393418302385,
  'y': -0.4073469917610737,
  'z': -0.39986031775197262}}

In [15]:
# %%timeit -- a test run of this with 1999 TNOs and 100 nights (63472 visits) from baseline2018b.db == 
#   11min 36s ± 8.54 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
# Generate the actual observations (using the linear interpolations) and write to output file.
tstep = 2.0 / 24.0  # tstep in units of days
obsFile = '_'.join([opsimrun, orbitfile.replace('.des', '').replace('.s3m', ''), 'obs.txt'])
print('Will write output to %s' % obsFile)
obs.run(simdata, obsFile, tstep=tstep)

Will write output to baseline2018b_neos_100_obs.txt
Generating ephemerides on a grid of 0.083333 day timesteps, then will extrapolate to opsim times.


In [16]:
# This is what the output file looks like:
!head -3 $obsFile

objId geo_dist ra dec magV dradt ddecdt velocity phase solarelon helio_dist h_lon h_lat true_anomaly time observationStartMJD night fieldRA fieldDec rotSkyPos filter visitExposureTime seeingFwhmEff seeingFwhmGeom fiveSigmaDepth solarElong ra dec magFilter dmagColor dmagTrail dmagDetect 
S1000001 1.5521554328 335.661569024 -4.66871691471 24.2367756326 -0.127241661894 -0.099168763745 0.161322698427 12.9028713254 146.840867277 2.45213370101 348.541969668 3.21849975573 176.458817876 59854.0760648 59854.0760648 2 335.151362 -5.689996 228.530394116 y 30.0 0.829128431145 0.733543570401 22.0670183628 145.792553804 335.151362 -5.689996 23.8294286408 -0.407346991761 0.0231607239649 0.0169491317618 
S1000001 1.55221035527 335.660588787 -4.66947882752 24.2369589087 -0.127297535485 -0.0991198719151 0.161336637608 12.9058864785 146.832227523 2.45213410577 348.543863385 3.21837682325 176.460699779 59854.0837384 59854.0837384 2 336.65378 -3.317561 242.063032863 y 30.0 0.777920585724 0.691450721465 22.